# How to Query OpenTargets Genetics for L2G Scores

### 1. Create Google Cloud Credentials

This step explains how to setup a Google Cloud Service Account to use Google BigQuery. Querying the first 1TB/month of public BigQuery is free, but there are fees beyond that, so you will need to link payment information to your credentials to run queries. Unless you are doing very large queries, you should not reach the 1TB limit.

1. Go to https://console.cloud.google.com/
2. If you do not have a project already, create one.
3. Go to https://console.cloud.google.com/iam-admin/serviceaccounts & click `+ CREATE SERVICE ACCOUNT`
4. Enter a descriptive Service Account name like "groberts-query-opentargets" & set the role to "owner"
5. Once back on the Service Accounts main page, click the "Actions" menu button > "Manage Keys"
6. `ADD KEY` > `Create New Key` > Select`JSON`.
7. A `.json` file will automatically download. **Be careful with your `.json`. You will need to call this file, but DO NOT upload it to GitHub as these are your private credentials and malicious actors could use your credentials to rack up expensive queries**

Once your service account is setup, the query below shows you how to access data in the BigQuery database
`bigquery-public-data.open_targets_genetics`.  You can view the table schema in the OpenTargets Genetics database by going to 
https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=open_targets_genetics&page=dataset .

Note that there is a separate database for the OpenTargets Platform `bigquery-public-data.open_targets_platform` that contains additional information that you may wish to map to the OpenTargets Genetics data you are pulling in, for example known drugs that target GWAS genes.

## 2. Setup to Run a BigQuery

Setup the BigQuery client

In [1]:
# Setup credentials for BigQuery (.json generated in 7 steps above)

from google.cloud import bigquery
credentials_path = '/home/robertg1/.ssh/test-bigquery-ot-956f8a01208f.json' #Do not push this file to GitHub!
client = bigquery.Client.from_service_account_json(credentials_path)

In [2]:
# Basic Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def process_json_format_l2g_columns(df, col="trait_efos"):
    df[col] = df[col].apply(lambda x: x["list"][0]["element"] if len(x["list"]) == 1 else ", ".join([i["element"] for i in x["list"]]))
    return df

## 3. Do an Initial Query to Identify All Available Lipid GWAS to Pull In

In [4]:
query_studies = '''
SELECT *
FROM `bigquery-public-data.open_targets_genetics.studies`
'''
# Note - as defined, this query returns 
query_studies_job = client.query(query_studies)

# Convert the query results to a Pandas dataframe
studies_query_res = query_studies_job.to_dataframe()

In [5]:
#get trait_efos in a simpler format for querying
studies = process_json_format_l2g_columns(studies_query_res.copy())

#query the lipid EFOs that we want
efo_ids = ["EFO_0004611", "EFO_0004612", "EFO_0004530"]
studies_of_interest = studies.query("trait_efos in @efo_ids")

In [6]:
#filter to some high-quality studies in the last 10 years

min_pub_date = pd.to_datetime("2014-01-01")
studies_of_interest.loc[:, "pub_date"] = pd.to_datetime(studies_of_interest["pub_date"])
studies_to_pull = (
    studies_of_interest.query("trait_efos in @efo_ids and num_assoc_loci >= 20 and pub_date >= @min_pub_date and not trait_reported.str.contains('\[|drinker\]', case=False)", engine="python")
    .sort_values(by="n_initial", ascending=False)
    .groupby(["pmid", "trait_efos"])
    .head(1)  # Keep only the top row in each group (optional)
    .reset_index()
)
print(f"There are {studies_to_pull.shape[0]} unique studies in this query")

There are 39 unique studies in this query


In [7]:
#define a few extra and add to the list
finngen = ["E4_HYPERCHOL", "E4_HYPERLIPNAS", "E4_HYPERLIPMIX"]
ukb = ["SAIGE_272_1", "SAIGE_272_11"]
all_gwas_to_pull = list(studies_to_pull["study_id"]) + finngen + ukb

## 4. Use the Study IDs to Perform the Full Query

Here, we will take the study IDs selected above and use those to query all genome-wide significant loci, all available co-localization data, and all available L2G scores for those studies.

In [8]:
# Optional: Define a list of GWAS Catalog IDs/SAIGE/FINNGEN/NEALE2 GWAS IDs to keep
#           This can be useful in selecting the highest quality/latest GWAS for each trait.
study_ids_to_keep = all_gwas_to_pull

# Construct parameterized query
query = '''
WITH ranked_genes AS (
  SELECT locus2gene.study_id, 
       locus2gene.chrom, locus2gene.pos, locus2gene.ref, locus2gene.alt, 
       study_metadata.*, 
       locus2gene.gene_id, genes.gene_name, locus2gene.y_proba_full_model,
       coloc.right_study, coloc.right_type, coloc.right_bio_feature, coloc.right_phenotype,
       coloc.coloc_h0, coloc.coloc_h1, coloc.coloc_h2, coloc.coloc_h3, coloc.coloc_h4,
       coloc.coloc_h4_h3, coloc.coloc_log2_h4_h3,
       lead_variants.pval,
       ROW_NUMBER() OVER(PARTITION BY locus2gene.study_id, genes.gene_name, coloc.right_study,
       coloc.right_bio_feature ORDER BY lead_variants.pval) AS rn
   
  FROM `bigquery-public-data.open_targets_genetics.locus2gene` AS locus2gene
   
  -- Get lead variant P-values
  INNER JOIN `bigquery-public-data.open_targets_genetics.variant_disease` AS lead_variants
    ON locus2gene.pos = lead_variants.lead_pos
      AND locus2gene.chrom = lead_variants.lead_chrom
      AND locus2gene.study_id = lead_variants.study_id
   
  -- Get colocoalization stats
  LEFT JOIN `bigquery-public-data.open_targets_genetics.variant_disease_coloc` AS coloc
    ON (locus2gene.gene_id = coloc.right_gene_id
      AND lead_variants.lead_chrom = coloc.left_chrom
      AND lead_variants.lead_pos = coloc.left_pos
      AND lead_variants.lead_ref = coloc.left_ref
      AND lead_variants.lead_alt = coloc.left_alt
      AND lead_variants.study_id = coloc.left_study
      AND coloc.right_type IN ('eqtl', 'pqtl'))

  -- Get GWAS metadata
  INNER JOIN `bigquery-public-data.open_targets_genetics.studies` AS study_metadata
    ON locus2gene.study_id = study_metadata.study_id
   
  -- Get HGNC IDs
  INNER JOIN `bigquery-public-data.open_targets_genetics.genes` AS genes
    ON locus2gene.gene_id = genes.gene_id
   
   -- Filter on the study IDs in the list study_ids_to_keep
   WHERE locus2gene.study_id IN UNNEST(@study_ids_to_keep)
   
   -- Remove the "raw" Neale lab results -- I'm not sure what this is
   AND locus2gene.study_id NOT LIKE '%raw%'
)

SELECT * FROM ranked_genes WHERE rn = 1;
'''

# Set query parameters
job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ArrayQueryParameter("study_ids_to_keep", "STRING", study_ids_to_keep)
    ]
)

# Run the query
# Note - as defined, this query returns 
query_job = client.query(query, job_config=job_config)

# Convert the query results to a Pandas dataframe
l2g = query_job.to_dataframe()
l2g.sort_values(by=['study_id', 'pval'])

,study_id,chrom,pos,ref,alt,study_id_1,ancestry_initial,ancestry_replication,n_cases,n_initial,...,right_phenotype,coloc_h0,coloc_h1,coloc_h2,coloc_h3,coloc_h4,coloc_h4_h3,coloc_log2_h4_h3,pval,rn
6949,GCST002897,11,116778201,G,C,GCST002897,{'list': [{'element': 'European=62166'}]},{'list': []},<NA>,62166,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000e-157,1
15006,GCST002897,11,116778201,G,C,GCST002897,{'list': [{'element': 'European=62166'}]},{'list': []},<NA>,62166,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000e-157,1
16734,GCST002897,11,116778201,G,C,GCST002897,{'list': [{'element': 'European=62166'}]},{'list': []},<NA>,62166,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000e-157,1
30520,GCST002897,11,116778201,G,C,GCST002897,{'list': [{'element': 'European=62166'}]},{'list': []},<NA>,62166,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000e-157,1
36507,GCST002897,11,116778201,G,C,GCST002897,{'list': [{'element': 'European=62166'}]},{'list': []},<NA>,62166,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000e-157,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83500,SAIGE_272_11,11,65705287,A,G,SAIGE_272_11,{'list': [{'element': 'European=406276'}]},{'list': []},33242,406276,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.490000e-08,1
83502,SAIGE_272_11,11,65705287,A,G,SAIGE_272_11,{'list': [{'element': 'European=406276'}]},{'list': []},33242,406276,...,ENSG00000213445,0.001266,0.032150,0.001497,0.037081,0.928007,25.026268,4.645371,3.490000e-08,1
85291,SAIGE_272_11,11,65705287,A,G,SAIGE_272_11,{'list': [{'element': 'European=406276'}]},{'list': []},33242,406276,...,ENSG00000172922,0.007134,0.181186,0.006094,0.154124,0.651462,4.226866,2.079588,3.490000e-08,1
85293,SAIGE_272_11,11,65705287,A,G,SAIGE_272_11,{'list': [{'element': 'European=406276'}]},{'list': []},33242,406276,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.490000e-08,1


In [9]:
l2g.to_csv('OpenTargets_L2G_Coloc_Lipids.noQC.6May24.csv.gz', compression="gzip", index=False)

## 4. Get the Credible Sets Associated with Each Locus

In [10]:
cred_set_query = '''
SELECT credible_sets.study_id, credible_sets.lead_variant_id, credible_sets.tag_variant_id, credible_sets.postprob, credible_sets.is95_credset
FROM `bigquery-public-data.open_targets_genetics.variant_disease_credset` AS credible_sets
WHERE credible_sets.study_id IN UNNEST(@study_ids_to_keep)
AND credible_sets.is95_credset = true;
'''

# Run the query
# Note - as defined, this query returns 
cred_set_query_job = client.query(cred_set_query, job_config=job_config)

# Convert the query results to a Pandas dataframe
cred_sets = cred_set_query_job.to_dataframe()
cred_sets

,study_id,lead_variant_id,tag_variant_id,postprob,is95_credset
0,SAIGE_272_1,8:58407830:A:G,8:58420135:G:A,0.052411,True
1,SAIGE_272_1,3:136402758:A:C,3:136384425:C:T,0.017859,True
2,SAIGE_272_1,5:157051633:C:T,5:158443054:A:T,0.002977,True
3,SAIGE_272_1,8:19967156:C:T,8:18405036:T:C,0.001851,True
4,SAIGE_272_1,19:11081053:C:T,19:11079976:G:A,0.024773,True
...,...,...,...,...,...
21312,SAIGE_272_11,19:11082415:T:G,19:11079398:G:A,0.049589,True
21313,SAIGE_272_11,16:72067626:C:G,16:72041694:A:T,0.217111,True
21314,SAIGE_272_11,19:11082415:T:G,19:11091630:G:T,0.016592,True
21315,SAIGE_272_11,8:58423997:C:T,8:58418723:T:G,0.061966,True


In [11]:
def get_credset_intervals(df):
    df['position'] = df['tag_variant_id'].str.split(':', expand=True)[1].astype(int)
    grouped = df.groupby('lead_variant_id')

    df['cred_set_min_pos'] = grouped['position'].transform('min')
    df['cred_set_max_pos'] = grouped['position'].transform('max')
    df['cred_set_size_bp'] = df['cred_set_max_pos'] - df['cred_set_min_pos']
    df['credset_n_variants'] = grouped['tag_variant_id'].transform('nunique')
    df=df.sort_values(by=["study_id", "lead_variant_id", "postprob"], ascending=[True, True, False])
    return(df)

cred_sets_annotated = get_credset_intervals(cred_sets.copy())
cred_sets_annotated

,study_id,lead_variant_id,tag_variant_id,postprob,is95_credset,position,cred_set_min_pos,cred_set_max_pos,cred_set_size_bp,credset_n_variants
12246,GCST90002412,10:100315722:G:A,10:100315722:G:A,0.999993,True,100315722,100315722,100315722,0,1
2285,GCST90002412,10:112161596:G:A,10:112161596:G:A,0.165411,True,112161596,112150963,112189906,38943,21
7105,GCST90002412,10:112161596:G:A,10:112180571:T:C,0.121886,True,112180571,112150963,112189906,38943,21
13220,GCST90002412,10:112161596:G:A,10:112174626:G:C,0.071861,True,112174626,112150963,112189906,38943,21
1164,GCST90002412,10:112161596:G:A,10:112161401:T:A,0.061224,True,112161401,112150963,112189906,38943,21
...,...,...,...,...,...,...,...,...,...,...
21055,SAIGE_272_11,9:22076072:T:C,9:22124451:A:G,0.006718,True,22124451,22072265,22125504,53239,40
20970,SAIGE_272_11,9:22076072:T:C,9:22124631:A:G,0.006672,True,22124631,22072265,22125504,53239,40
21066,SAIGE_272_11,9:22076072:T:C,9:22115960:A:G,0.006066,True,22115960,22072265,22125504,53239,40
20574,SAIGE_272_11,9:22076072:T:C,9:22124478:A:G,0.005949,True,22124478,22072265,22125504,53239,40


# 5. Add V2G Scores for Each Credible Set Variant

In [12]:
unique_gene_ids = l2g["gene_id"].unique().tolist()
unique_tag_variants = cred_sets_annotated["tag_variant_id"].unique().tolist()
len(unique_tag_variants)

18847

In [13]:
v2g_query = '''
WITH v2g_concat AS (
  SELECT *,
         CONCAT(v2g.chr_id, ':', CAST(v2g.position AS STRING), ':', v2g.ref_allele, ':', v2g.alt_allele) AS variant_id
  FROM `bigquery-public-data.open_targets_genetics.variant_gene` AS v2g
)

SELECT variant_id,
       gene_id,
       CASE WHEN COUNT(DISTINCT fpred_max_label) = 1 THEN MAX(fpred_max_label) ELSE 'NA' END AS fpred_max_label,
       MAX(fpred_max_score) AS v2g_fpred_max_score,
       MAX(qtl_score_q) AS v2g_qtl_score_q,
       MAX(interval_score_q) AS v2g_interval_score_q,
       MAX(distance_score_q) AS v2g_distance_score_q,
       MIN(d) AS distance_to_tss
FROM v2g_concat
WHERE variant_id IN UNNEST (@unique_tag_variants)
GROUP BY variant_id, gene_id;

'''

# Run the query
# Note - as defined, this query returns 
# Set query parameters
v2g_job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ArrayQueryParameter("unique_tag_variants", "STRING", unique_tag_variants),
        bigquery.ArrayQueryParameter("unique_gene_ids", "STRING", unique_gene_ids)
    ]
)

# Convert the query results to a Pandas dataframe
v2g_query_job = client.query(v2g_query, job_config=v2g_job_config)
v2g = v2g_query_job.to_dataframe()
v2g

,variant_id,gene_id,fpred_max_label,v2g_fpred_max_score,v2g_qtl_score_q,v2g_interval_score_q,v2g_distance_score_q,distance_to_tss
0,16:31070704:T:C,ENSG00000099377,NA,NaN,0.9,0.7,0.8,85497
1,16:31068513:C:A,ENSG00000099377,NA,NaN,0.9,0.7,0.8,83306
2,1:150966095:G:A,ENSG00000117360,NA,NaN,0.0,NaN,NaN,<NA>
3,16:31079086:G:C,ENSG00000099377,NA,NaN,0.9,0.8,0.8,93879
4,16:31057070:G:A,ENSG00000099377,NA,NaN,0.9,0.6,0.9,71863
...,...,...,...,...,...,...,...,...
259823,5:123586079:A:G,ENSG00000151304,NA,NaN,NaN,0.4,NaN,<NA>
259824,2:65052280:A:G,ENSG00000143952,NA,NaN,NaN,0.4,NaN,<NA>
259825,1:150399540:T:G,ENSG00000143379,NA,NaN,NaN,0.0,NaN,<NA>
259826,2:218546480:A:G,ENSG00000187736,NA,NaN,NaN,0.0,NaN,<NA>


In [14]:
v2g.to_csv('OpenTargets_Variant2GeneScores_Lipids.6May24.csv', index=False)

## 6. Perform Joins and Cleanup Dataframes

In [15]:
filtered_l2g = process_json_format_l2g_columns(l2g.copy())

#create the variant IDs
filtered_l2g["lead_variant_id"] = filtered_l2g.chrom.astype(str) + ':' + filtered_l2g.pos.astype(str) + ':' + filtered_l2g.ref.astype(str) + ':' + filtered_l2g.alt.astype(str)

#for continuous traits, fill n_cases with n_initial
filtered_l2g["n_cases"] = filtered_l2g["n_cases"].fillna(filtered_l2g["n_initial"])

#rename some columns
filtered_l2g.rename(columns={'y_proba_full_model': 'L2G',
                             'pval' : 'lead_variant_pval',
                             'right_study' : 'coloc_study',
                             'right_type' : 'coloc_type',
                             'right_bio_feature' : 'coloc_tissue',
                             'right_phenotype' : 'coloc_probe_transcript_id'},
                    inplace=True)

filtered_l2g = filtered_l2g.sort_values(by=['trait_efos', 'study_id', 'lead_variant_pval', 'L2G', 'coloc_h4'],
                                        ascending=[True, True, True, False, False])

#filter down the columns to make this df a bit easier to work with
desired_columns = ['trait_efos', 'trait_reported',
                   'source', 'study_id', 'pmid',
                   'pub_date', 'n_cases','num_assoc_loci',
                   'lead_variant_id', 'lead_variant_pval',
                   'gene_name', 'L2G',
                   'coloc_study', 'coloc_type', 'coloc_tissue', 'coloc_probe_transcript_id',
                   'coloc_h0', 'coloc_h1', 'coloc_h2', 'coloc_h3', 'coloc_h4',
                  'coloc_h4_h3', 'coloc_log2_h4_h3']
filtered_l2g = filtered_l2g[desired_columns].drop_duplicates()
filtered_l2g

,trait_efos,trait_reported,source,study_id,pmid,pub_date,n_cases,num_assoc_loci,lead_variant_id,lead_variant_pval,...,coloc_type,coloc_tissue,coloc_probe_transcript_id,coloc_h0,coloc_h1,coloc_h2,coloc_h3,coloc_h4,coloc_h4_h3,coloc_log2_h4_h3
32231,EFO_0003774,Hyperlipidemia,SAIGE,SAIGE_272_1,None,2018-10-24,35844,45,19:44908822:C:T,2.100000e-72,...,pqtl,UBERON_0001969,APOE_5312_49_3,1.080201e-60,6.397096e-56,1.699519e-08,0.000006,0.999993,154213.068110,17.234565
86974,EFO_0003774,Hyperlipidemia,SAIGE,SAIGE_272_1,None,2018-10-24,35844,45,19:44908822:C:T,2.100000e-72,...,eqtl,NEUTROPHIL_CD15,ILMN_1683475,2.650833e-53,4.170651e-01,4.167357e-54,0.065049,0.517886,7.961524,2.993045
17559,EFO_0003774,Hyperlipidemia,SAIGE,SAIGE_272_1,None,2018-10-24,35844,45,19:44908822:C:T,2.100000e-72,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32230,EFO_0003774,Hyperlipidemia,SAIGE,SAIGE_272_1,None,2018-10-24,35844,45,19:44908822:C:T,2.100000e-72,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45234,EFO_0003774,Hyperlipidemia,SAIGE,SAIGE_272_1,None,2018-10-24,35844,45,19:44908822:C:T,2.100000e-72,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44395,HP_0003124,Hypercholesterolemia,SAIGE,SAIGE_272_11,None,2018-10-24,33242,40,11:65705287:A:G,3.490000e-08,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25413,HP_0003124,Hypercholesterolemia,SAIGE,SAIGE_272_11,None,2018-10-24,33242,40,11:65705287:A:G,3.490000e-08,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85293,HP_0003124,Hypercholesterolemia,SAIGE,SAIGE_272_11,None,2018-10-24,33242,40,11:65705287:A:G,3.490000e-08,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76516,HP_0003124,Hypercholesterolemia,SAIGE,SAIGE_272_11,None,2018-10-24,33242,40,11:65705287:A:G,3.490000e-08,...,eqtl,PLATELET,ILMN_1652777,2.531117e-02,6.438328e-01,4.154343e-03,0.105452,0.221250,2.098121,1.069098


In [16]:
filtered_l2g.to_csv('OpenTargets_L2G_Coloc_Lipids.Cleaned.6May24.csv', index=False)

In [17]:
cred_sets_final=cred_sets_annotated.merge(v2g, how="left", left_on="tag_variant_id", right_on="variant_id")
cred_sets_final=cred_sets_final.query('distance_to_tss<=500000', engine='python')
cred_sets_final

,study_id,lead_variant_id,tag_variant_id,postprob,is95_credset,position,cred_set_min_pos,cred_set_max_pos,cred_set_size_bp,credset_n_variants,variant_id,gene_id,fpred_max_label,v2g_fpred_max_score,v2g_qtl_score_q,v2g_interval_score_q,v2g_distance_score_q,distance_to_tss
0,GCST90002412,10:100315722:G:A,10:100315722:G:A,0.999993,True,100315722,100315722,100315722,0,1,10:100315722:G:A,ENSG00000120054,NA,NaN,NaN,NaN,0.5,233853
1,GCST90002412,10:100315722:G:A,10:100315722:G:A,0.999993,True,100315722,100315722,100315722,0,1,10:100315722:G:A,ENSG00000107566,NA,NaN,NaN,NaN,0.7,129689
2,GCST90002412,10:100315722:G:A,10:100315722:G:A,0.999993,True,100315722,100315722,100315722,0,1,10:100315722:G:A,ENSG00000107593,intron_variant,0.1,0.8,NaN,1.0,14542
4,GCST90002412,10:100315722:G:A,10:100315722:G:A,0.999993,True,100315722,100315722,100315722,0,1,10:100315722:G:A,ENSG00000196072,NA,NaN,NaN,NaN,0.9,29042
5,GCST90002412,10:100315722:G:A,10:100315722:G:A,0.999993,True,100315722,100315722,100315722,0,1,10:100315722:G:A,ENSG00000166135,NA,NaN,NaN,0.0,0.6,213350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294531,SAIGE_272_11,9:22076072:T:C,9:22124478:A:G,0.005949,True,22124478,22072265,22125504,53239,40,9:22124478:A:G,ENSG00000147883,NA,NaN,NaN,0.6,0.8,115173
294532,SAIGE_272_11,9:22076072:T:C,9:22125504:G:C,0.005790,True,22125504,22072265,22125504,53239,40,9:22125504:G:C,ENSG00000147889,NA,NaN,NaN,0.3,0.7,130203
294533,SAIGE_272_11,9:22076072:T:C,9:22125504:G:C,0.005790,True,22125504,22072265,22125504,53239,40,9:22125504:G:C,ENSG00000147883,NA,NaN,NaN,0.6,0.8,116199
294534,SAIGE_272_11,9:22076072:T:C,9:22125504:G:C,0.005790,True,22125504,22072265,22125504,53239,40,9:22125504:G:C,ENSG00000176399,NA,NaN,NaN,NaN,0.4,321320


In [18]:
def compute_weighted_mean(df,
                          weights = {
    'v2g_fpred_max_score': 1,
    'v2g_qtl_score_q': 0.66,
    'v2g_distance_score_q': 0.33,
    'v2g_interval_score_q': 0.33
}):
    
    df.fillna(0, inplace=True)
    
    # Compute the weighted mean score
    weighted_sum = sum(df[col] * weights[col] for col in weights)
    total_weight = sum(weights.values())
    weighted_mean = weighted_sum / total_weight

    # Add the weighted mean score as a new column named 'v2g'
    df['v2g'] = weighted_mean
    
    return df

In [19]:
compute_weighted_mean(df=cred_sets_final)

,study_id,lead_variant_id,tag_variant_id,postprob,is95_credset,position,cred_set_min_pos,cred_set_max_pos,cred_set_size_bp,credset_n_variants,variant_id,gene_id,fpred_max_label,v2g_fpred_max_score,v2g_qtl_score_q,v2g_interval_score_q,v2g_distance_score_q,distance_to_tss,v2g
0,GCST90002412,10:100315722:G:A,10:100315722:G:A,0.999993,True,100315722,100315722,100315722,0,1,10:100315722:G:A,ENSG00000120054,NA,0.0,0.0,0.0,0.5,233853,0.071121
1,GCST90002412,10:100315722:G:A,10:100315722:G:A,0.999993,True,100315722,100315722,100315722,0,1,10:100315722:G:A,ENSG00000107566,NA,0.0,0.0,0.0,0.7,129689,0.099569
2,GCST90002412,10:100315722:G:A,10:100315722:G:A,0.999993,True,100315722,100315722,100315722,0,1,10:100315722:G:A,ENSG00000107593,intron_variant,0.1,0.8,0.0,1.0,14542,0.412931
4,GCST90002412,10:100315722:G:A,10:100315722:G:A,0.999993,True,100315722,100315722,100315722,0,1,10:100315722:G:A,ENSG00000196072,NA,0.0,0.0,0.0,0.9,29042,0.128017
5,GCST90002412,10:100315722:G:A,10:100315722:G:A,0.999993,True,100315722,100315722,100315722,0,1,10:100315722:G:A,ENSG00000166135,NA,0.0,0.0,0.0,0.6,213350,0.085345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294531,SAIGE_272_11,9:22076072:T:C,9:22124478:A:G,0.005949,True,22124478,22072265,22125504,53239,40,9:22124478:A:G,ENSG00000147883,NA,0.0,0.0,0.6,0.8,115173,0.199138
294532,SAIGE_272_11,9:22076072:T:C,9:22125504:G:C,0.005790,True,22125504,22072265,22125504,53239,40,9:22125504:G:C,ENSG00000147889,NA,0.0,0.0,0.3,0.7,130203,0.142241
294533,SAIGE_272_11,9:22076072:T:C,9:22125504:G:C,0.005790,True,22125504,22072265,22125504,53239,40,9:22125504:G:C,ENSG00000147883,NA,0.0,0.0,0.6,0.8,116199,0.199138
294534,SAIGE_272_11,9:22076072:T:C,9:22125504:G:C,0.005790,True,22125504,22072265,22125504,53239,40,9:22125504:G:C,ENSG00000176399,NA,0.0,0.0,0.0,0.4,321320,0.056897


In [20]:
cred_sets_final.to_csv('OpenTargets_Credible_Sets_Lipids.6May24.csv', index=False)

In [21]:
# Description of each column
column_descriptions = {
    'trait_efos': 'EFO Trait/Disase Codes. In this study, only lipid-related codes EFO_0003774, EFO_0004530, EFO_0004611, EFO_0004612, and HP_0003124 are included.',
    'trait_reported': 'Reported trait for the GWAS (e.g. Hyperlipidemia)',
    'source': 'Source of the study data. Could be SAIGE (UKB) or GCST (GWAS Catalog)',
    'study_id': 'ID of the study. There are 39 GWAS catalog studies of lipid traits from the last 10 years + 2 studies from UKB conducted with SAIGE',
    'pmid': 'PubMed ID of the study.',
    'pub_date': 'Publication date of the study.',
    'n_cases': 'Number of cases in the study.',
    'num_assoc_loci': 'Number of independent associated loci in the study.',
    'lead_variant_id': 'ID of the lead variant for each independent locus.',
    'lead_variant_pval': 'P-value of the lead variant in the study.',
    'gene_name': 'Gene Name wtihin 500Kb of lead_variant',
    'L2G': 'OpenTargets Locus-to-gene score associated with the locus and the gene.',
    'coloc_study': 'Colocalization study associated with the variant. Note that colocalization was only attempted if there were overlaps in the credible sets.',
    'coloc_type': 'Type of colocalization study (eQTL or pQTL)',
    'coloc_tissue': 'Tissue type used in colocalization study.',
    'coloc_probe_transcript_id': 'Transcript ID of the probe used in colocalization study.',
    'coloc_h0': 'Colocalization hypothesis H0.',
    'coloc_h1': 'Colocalization hypothesis H1.',
    'coloc_h2': 'Colocalization hypothesis H2.',
    'coloc_h3': 'Colocalization hypothesis H3.',
    'coloc_h4': 'Colocalization hypothesis H4.',
    'coloc_h4_h3': 'Ratio of colocalization hypotheses H4 to H3.',
    'coloc_log2_h4_h3': 'Log2 ratio of colocalization hypotheses H4 to H3.',
    'postprob': 'Posterior probability of the tag variant being causal.',
    'is95_credset': 'Indicator if the tag_variant_id is in the 95% credible set for the locus.',
    'cred_set_min_pos': 'Minimum position of variants within the credible set for this locus.',
    'cred_set_max_pos': 'Maximum position of variants within the credible set for this locus.',
    'cred_set_size_bp': 'Size of the credible set in base pairs.',
    'credset_n_variants': 'Number of variants within the credible set for this locus.',
    'gene_id' : 'Ensmbl gene ID',
    'fpred_max_label': 'Most severe VEP consequence of the tag_variant_id on the gene_id',
    'v2g_fpred_max_score' : 'Score based on VEP functional prediction as described at https://github.com/opentargets/v2g_data/blob/master/configs/vep_consequences.tsv',
    'v2g_qtl_score_q': 'Maximum quantile of OpenTargets V2G based on -log10(QTL p-value)',
    'v2g_interval_score_q': 'Maximum quantile of OpenTargets V2G based on PCHi-C from Javierre et al. (Cell, 2016), Enhancer-TSS correlation from Andersson et al. (Nature, 2014) or DHS-promoter correlation from Thurman et al. (Nature,  2012)',
    'v2g_distance_score_q': 'Maximum quantile of OpenTargets V2G based on 1/distance to tss',
    'v2g': 'Weighted mean across the v2g quantiles, as described in https://genetics-docs.opentargets.org/our-approach/data-pipeline',
    'distance_to_tss' : 'distance to gene transcription start site (TSS) in bp'
}

# Define credible set related columns
credible_set_cols = [
    'tag_variant_id', 'postprob',
    'is95_credset', 'cred_set_min_pos', 'cred_set_max_pos',
    'cred_set_size_bp', 'credset_n_variants',
    'gene_id','fpred_max_label', 'v2g_fpred_max_score', 'v2g_qtl_score_q',
    'v2g_interval_score_q', 'v2g_distance_score_q', 'v2g', 'distance_to_tss'
]

# Define other columns
other_cols = set(column_descriptions.keys()) - set(credible_set_cols)

# Write descriptions to README.txt
with open('README.txt', 'w') as readme_file:
  readme_file.write("\n# These two files can be joined on study_id and lead_variant_id \n\n")
  readme_file.write("# Credible Set File Columns\n\n")
  for col in credible_set_cols:
    readme_file.write(f"{col}: {column_descriptions.get(col, 'Description not available.')}\n")

  readme_file.write("\n# Other File Columns\n\n")
  for col in sorted(other_cols):
    readme_file.write(f"{col}: {column_descriptions.get(col, 'Description not available.')}\n")